[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sonder-art/fdd_p25/blob/main/professor/intro_python_interactivo/notebooks/05b_Alcance_y_Params_Avanzados.ipynb)


# 05b · Alcance (LEGB) y Parámetros Avanzados — comprensión profunda

Objetivo: dominar cómo Python pasa y resuelve nombres (LEGB), entender mutabilidad vs reasignación y copias vs alias, y aprender patrones avanzados de firmas (*args/**kwargs). También veremos lambdas/closures (late binding) y una nota de recursión. Avanzaremos paso a paso con explicaciones detalladas y código comentado.


## 1) Passing semantics — primeros pasos

Vamos a diferenciar qué ocurre al pasar inmutables (int/str/tuple) y mutables (list/dict/set). Lo mostraremos con “antes/después” y comentarios dentro del código.


In [1]:
# Inmutable: reasignar dentro NO afecta al llamador

def toca_int(x: int) -> None:
    # Dentro, x es un nombre local. Reasignarlo no cambia fuera.
    x = x + 10
    print("(func) x local:", x)

n = 5
print("(antes) n:", n)
toca_int(n)
print("(después) n:", n)

# Mutable: mutar SÍ afecta al llamador

def agrega_elemento(xs: list[int]) -> None:
    # Mutación in-place: esto cambia el mismo objeto que ve el llamador
    xs.append(99)
    print("(func) xs local:", xs)

arr = [1, 2]
print("(antes) arr:", arr)
agrega_elemento(arr)
print("(después) arr:", arr)



(antes) n: 5
(func) x local: 15
(después) n: 5
(antes) arr: [1, 2]
(func) xs local: [1, 2, 99]
(después) arr: [1, 2, 99]


## 2) Reasignar nombre vs mutar objeto

- Reasignar un nombre local crea un nuevo vínculo (no afecta afuera).
- Mutar un objeto cambia el estado del mismo objeto referenciado.


In [2]:
def reasigna_lista(xs: list[int]) -> None:
    # Reasignar el nombre local 'xs' NO afecta la variable del llamador
    xs = [0]  # nuevo objeto, solo dentro de la función
    print("(func) xs reasignada:", xs)

def muta_lista(xs: list[int]) -> None:
    # Mutación in-place SÍ se refleja fuera
    xs.append(42)
    print("(func) xs mutada:", xs)

base = [1, 2, 3]
print("(antes) base:", base)
reasigna_lista(base)
print("(después reasigna) base:", base)
muta_lista(base)
print("(después muta) base:", base)



(antes) base: [1, 2, 3]
(func) xs reasignada: [0]
(después reasigna) base: [1, 2, 3]
(func) xs mutada: [1, 2, 3, 42]
(después muta) base: [1, 2, 3, 42]


## 3) Copias vs alias (recordatorio breve)

- Alias: `b = a` apunta al MISMO objeto (mutar en un lado afecta el otro).
- Copia superficial (shallow): `a.copy()`, `list(a)`, `a[:]` crean un NUEVO contenedor, pero sub‑objetos siguen compartidos.
- Copia profunda (deepcopy): duplica recursivamente sub‑objetos (útil con anidados).


In [3]:
a = [1, [2, 3]]
b = a              # alias
c = a[:]           # shallow copy

# Mutación de sub‑lista afecta alias y copia superficial
a[1].append(4)
print("a:", a)
print("b (alias):", b)
print("c (shallow):", c)

import copy
d = copy.deepcopy(a)
a[1].append(5)
print("d (deepcopy):", d)



a: [1, [2, 3, 4]]
b (alias): [1, [2, 3, 4]]
c (shallow): [1, [2, 3, 4]]
d (deepcopy): [1, [2, 3, 4]]


## 4) LEGB — resolución de nombres (muy detallado)

Orden de búsqueda de nombres: Local → Enclosing → Global → Builtins. Vamos a verlo en capas, con ejemplos breves y comentarios explícitos.


In [4]:
# 4.1 Local vs Global
X = "global"

def muestra():
    # Python busca X: Local (no), luego Global (sí)
    print("X visto dentro:", X)

print("X global:", X)
muestra()



X global: global
X visto dentro: global


In [5]:
# 4.2 Enclosing: función dentro de función (lectura)

def externa():
    mensaje = "enclosing"
    def interna():
        # No hay 'mensaje' local, sube al enclosing y lo encuentra
        print("mensaje visto por interna:", mensaje)
    interna()

externa()



mensaje visto por interna: enclosing


In [6]:
# 4.3 nonlocal: modificar variable del enclosing

def fabrica_contador(inicial=0):
    cuenta = inicial  # variable en el enclosing
    def sumar():
        nonlocal cuenta        # indica que queremos modificar la del enclosing
        cuenta += 1            # sin nonlocal, esto crearía un nombre local y fallaría
        return cuenta
    return sumar

c = fabrica_contador(10)
print(c())
print(c())



11
12


In [7]:
# 4.4 global: evita su uso salvo casos muy puntuales

Y = 0

def toca_global():
    global Y
    Y = 99         # rebind en el espacio global (difícil de razonar y testear)

print("Y antes:", Y)
toca_global()
print("Y después:", Y)



Y antes: 0
Y después: 99


### 4.5 Edge cases de LEGB (sombreado y “cuándo” se evalúa)

- Sombreado (shadowing): un nombre local con el mismo identificador “tapa” al externo en ese bloque.
- Cierres (closures) y tiempos: la función interna ve la variable por nombre; según el patrón, puede evaluarse más tarde (ej. en lambdas dentro de bucles).


In [8]:
# Sombreado: el nombre local 'X' oculta al global dentro del bloque
X = "global"

def sombra():
    X = "local"   # sombreado
    print("dentro:", X)  # muestra 'local'

sombra()
print("fuera:", X)       # sigue siendo 'global'

# Timing con cierre (closure) — se evalúa al usarlo
funcs = []
for i in range(3):
    def f():
        # f usa 'i' por nombre; todas verán el último valor si no fijamos i
        return i
    funcs.append(f)
print([fn() for fn in funcs])  # late binding clásico

# Solución: fijar i como arg por defecto
funcs2 = []
for i in range(3):
    def f(i=i):
        return i
    funcs2.append(f)
print([fn() for fn in funcs2])



dentro: local
fuera: global
[2, 2, 2]
[0, 1, 2]


## 5) *args y **kwargs — avanzado y claro

Orden recomendado en la firma: posicionales, *args, keyword‑only (tras *), **kwargs. También veremos desempaque al llamar con * y **, y cómo inspeccionar lo recibido con prints etiquetados.


In [9]:
def firma(a, b=0, *args, escala=1, **kwargs):
    # Imprime todo lo recibido para inspeccionar
    print(f"a={a!r} b={b!r} args={args!r} escala={escala!r} kwargs={kwargs!r}")
    total = (a + b + sum(args)) * escala
    return total

print("llamada simple:", firma(1))
print("con *args:", firma(1, 2, 3, 4))
print("keyword-only:", firma(1, escala=10))
print("mezcla:", firma(1, 2, 3, escala=2, x=99))

# Desempaque al llamar con * y **
pos = (5, 6, 7)
nom = {"escala": 3, "extra": True}
print("desempaque:", firma(*pos, **nom))



a=1 b=0 args=() escala=1 kwargs={}
llamada simple: 1
a=1 b=2 args=(3, 4) escala=1 kwargs={}
con *args: 10
a=1 b=0 args=() escala=10 kwargs={}
keyword-only: 10
a=1 b=2 args=(3,) escala=2 kwargs={'x': 99}
mezcla: 12
a=5 b=6 args=(7,) escala=3 kwargs={'extra': True}
desempaque: 54


### Nota sobre f-strings: `!r`, `!s`, `!a` y formateo con `:`

- `f"{expr!r}"` aplica `repr(expr)` antes de incrustar el valor. `repr` es útil para depurar: muestra comillas, saltos de línea como `\n`, etc.
- `f"{expr!s}"` aplica `str(expr)` (es la conversión por defecto de `print`). Suele ser más “amigable” pero menos precisa para depurar.
- `f"{expr!a}"` aplica `ascii(expr)` (convierte a ASCII escapando caracteres no ASCII).
- Puedes combinar conversión con especificadores de formato usando `:`: por ejemplo `f"{num:08d}"` (entero con ceros a la izquierda) o `f"{pi:.3f}"` (3 decimales).

En la línea `print(f"a={a!r} b={b!r} args={args!r} escala={escala!r} kwargs={kwargs!r}")` usamos `!r` para ver las representaciones “fiables” (repr) de todos los argumentos, ideal para inspeccionar qué llegó realmente a la función.


## 6) Lambdas, cierres y late binding (con solución)

- Lambdas: funciones pequeñas “en línea”; úsalas cuando la simplicidad lo permita.
- Closure: una función interna puede “ver” nombres del enclosing.
- Late binding: lambdas en bucles capturan el nombre, no el valor; usa `i=i` para fijar el valor actual.


In [10]:
# Demostración breve con lambda y solución
fns = []
for i in range(3):
    fns.append(lambda: i)   # late binding: todas verán el último i
print("late binding:", [f() for f in fns])

fns_fix = []
for i in range(3):
    fns_fix.append(lambda i=i: i)  # fijamos el valor actual como default
print("fijado con i=i:", [f() for f in fns_fix])



late binding: [2, 2, 2]
fijado con i=i: [0, 1, 2]


## 7) Recursión — nota breve

- Siempre define un caso base claro y uno recursivo.
- Cuidado con la profundidad de recursión (por defecto ~1000 llamadas).
- Cuando el problema crece mucho, considera una versión iterativa u otras técnicas.


## 8) Resumen y enlaces

- Diferencias clave: inmutable vs mutable, reasignar vs mutar, alias vs copias.
- LEGB detallado: Local → Enclosing → Global → Builtins; nonlocal/global y sus efectos.
- Firmas avanzadas: *args/**kwargs, keyword‑only, desempaque.
- Lambdas/closures y late binding; solución con `i=i`.
- Recursión: caso base y límites prácticos.

Enlaces:
- Tutorial oficial (namespaces y scope)
- Módulo `copy` (deepcopy)
- Referencia a 03 (copias), 06 (comprehensions/map/filter), 07 (errores/raise), 08 (generadores/closures)
